In [1]:
from plumber.streams import IBLEidStream
from plumber.pipes import DaartInf, IBLMarkerExtractor, SimpleEnsembler, OverviewPlotter

from plumber import Stream, Pipe, PipeLine

artifact_path = "/home/nuttidalab/Documents/plab/plumber/artifacts"

Connected to https://openalyx.internationalbrainlab.org as user "intbrainlab"
Connected to https://openalyx.internationalbrainlab.org as user "intbrainlab"


In [ ]:
# # One stream
# input_stream = IBLEidStream('eid_stream', 2)

# # 3 pipes
# extractor = IBLMarkerExtractor('markers', artifact_path, smooth=False, mtype='LP')
# predictor_1 = DaartInf('Mod1', artifact_path, "mod1")
# predictor_2 = DaartInf('Mod2', artifact_path, "mod2")
# predictor_3 = DaartInf('Mod3', artifact_path, "mod3")
# predictor_4 = DaartInf('Mod4', artifact_path, "mod4")
# predictor_5 = DaartInf('Mod5', artifact_path, "mod5")
# ensembler = SimpleEnsembler('ens')
# plotter = OverviewPlotter('overview', artifact_path, mtype='LP', eid=input_stream.data["eid_stream"]["eid"])

# # 1 pipeline
# PL = PipeLine()
# PL.register(input_stream, extractor)

# PL.register(extractor, predictor_1)
# PL.register(extractor, predictor_2)
# PL.register(extractor, predictor_3)
# PL.register(extractor, predictor_4)
# PL.register(extractor, predictor_5)

# PL.register(predictor_1, ensembler)
# PL.register(predictor_2, ensembler)
# PL.register(predictor_3, ensembler)
# PL.register(predictor_4, ensembler)
# PL.register(predictor_5, ensembler)

# PL.register(ensembler, plotter)

# PL.flow(input_stream)

In [3]:
# PL.plot(title="IBL Paw Pipeline", figsize=(5, 5), dpi=100)

In [4]:
def make_pipeline(idx=0, marker_type = 'LP'):

    # One stream
    input_stream = IBLEidStream('eid_stream', idx)

    # 3 pipes
    extractor = IBLMarkerExtractor('markers', artifact_path, smooth=False, mtype='LP')
    predictor_1 = DaartInf('Mod1', artifact_path, "mod1")
    predictor_2 = DaartInf('Mod2', artifact_path, "mod2")
    predictor_3 = DaartInf('Mod3', artifact_path, "mod3")
    predictor_4 = DaartInf('Mod4', artifact_path, "mod4")
    predictor_5 = DaartInf('Mod5', artifact_path, "mod5")
    ensembler = SimpleEnsembler('ens')
    plotter = OverviewPlotter('overview', artifact_path, mtype='LP', eid=input_stream.data["eid_stream"]["eid"])

    # 1 pipeline
    pl = PipeLine()
    pl.register(input_stream, extractor)

    pl.register(extractor, predictor_1)
    pl.register(extractor, predictor_2)
    pl.register(extractor, predictor_3)
    pl.register(extractor, predictor_4)
    pl.register(extractor, predictor_5)

    pl.register(predictor_1, ensembler)
    pl.register(predictor_2, ensembler)
    pl.register(predictor_3, ensembler)
    pl.register(predictor_4, ensembler)
    pl.register(predictor_5, ensembler)

    pl.register(ensembler, plotter)

    pl.flow(input_stream)

    return input_stream
    

In [ ]:
for i in range(30, 40):
    input_stream = make_pipeline(i)
    print(i+1)